In [1]:
!pip3 install pulp


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [20]:
from pulp import *

def k_tsp_mtz_encoding(n, k, cost_matrix):
    # Check inputs are OK
    assert 1 <= k < n
    assert len(cost_matrix) == n, f'Cost matrix is not {n}x{n}'
    assert all(len(cj) == n for cj in cost_matrix), f'Cost matrix is not {n}x{n}'
    
    # Create the problem
    prob = LpProblem("kTSP", LpMinimize)
    
    # Decision variables
    x = {(i, j): LpVariable(f'x{i}{j}', cat='Binary') for i in range(n) for j in range(n) if i != j}
    spt = {i: LpVariable(f'spt{i}', lowBound=1, upBound=n, cat='Continuous') for i in range(1, n)}

    # Objective function
    prob += sum(cost_matrix[i][j] * x[(i, j)] for i in range(n) for j in range(n) if i != j and cost_matrix[i][j] is not None)

    # Constraints
    for i in range(n):
        if i == 0:
            prob += sum(x[(0, j)] for j in range(1, n) if cost_matrix[0][j] is not None) == k
            prob += sum(x[(j, 0)] for j in range(1, n) if cost_matrix[j][0] is not None) == k
        else:
            prob += sum(x[(i, j)] for j in range(n) if j != i and cost_matrix[i][j] is not None) == 1
            prob += sum(x[(j, i)] for j in range(n) if j != i and cost_matrix[j][i] is not None) == 1

    # Time stamp constraints
    for i in range(1, n):
        for j in range(1, n):
            if i != j and cost_matrix[i][j] is not None:
                prob += spt[i] - spt[j] + (n * x[(i, j)]) <= n - 1

    # Solve the problem
    prob.solve()

    # Construct the solution
    result = [[] for _ in range(k)]
    visited_nodes = set()
    curr_sp = 0
    curr_n = 0
    result[curr_sp].append(curr_n)
    visited_nodes.add(curr_n)

    while len(visited_nodes) < n:
        next_node = None
        min_cost = float('inf')
        for j in range(1, n):
            if cost_matrix[curr_n][j] is not None and x[(curr_n, j)].value() > 0.5 and j not in visited_nodes:
                if cost_matrix[curr_n][j] < min_cost:
                    next_node = j
                    min_cost = cost_matrix[curr_n][j]
        if next_node is not None:
            curr_n = next_node
            result[curr_sp].append(curr_n)
            visited_nodes.add(curr_n)
        else:
            # If no more edges can be found, move to the next salesperson
            curr_sp += 1
            if curr_sp < k:
                curr_n = 0
                result[curr_sp].append(curr_n)
                visited_nodes.add(curr_n)
            else:
                break

    # Calculate the total tour cost
    tour_cost = 0
    for tour in result:
        if tour:
            i = 0
            for j in tour[1:]:
                tour_cost += cost_matrix[i][j]
                i = j
            tour_cost += cost_matrix[i][0]

    return result

In [21]:
cost_matrix=[ [None,3,4,3,5],
             [1, None, 2,4, 1],
             [2, 1, None, 5, 4],
             [1, 1, 5, None, 4],
             [2, 1, 3, 5, None] ]
n=5
k=2
all_tours = k_tsp_mtz_encoding(n, k, cost_matrix)
print(f'Your code returned tours: {all_tours}')
assert len(all_tours) == k, f'k={k} must yield two tours -- your code returns {len(all_tours)} tours instead'

tour_cost = 0
for tour in all_tours:
    assert tour[0] == 0, 'Each salesperson tour must start from vertex 0'
    i = 0
    for j in tour[1:]:
        tour_cost += cost_matrix[i][j]
        i = j
    tour_cost += cost_matrix[i][0]

print(f'Tour cost obtained by your code: {tour_cost}')
assert abs(tour_cost - 12) <= 0.001, f'Expected tour cost is 12, your code returned {tour_cost}'
for i in range(1, n):
    is_in_tour = [ 1 if i in tour else 0 for tour in all_tours]
    assert sum(is_in_tour) == 1, f' vertex {i} is in {sum(is_in_tour)} tours -- this is incorrect'

print('test passed: 3 points')

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/josiahroa/workspace/cs-masters/5414-greedy-algorithms/karatsuba/.venv/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/3g/sywttvsx0m34y00rmp2v39b00000gn/T/d158203408d248b0b327bfa99a1bac16-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/3g/sywttvsx0m34y00rmp2v39b00000gn/T/d158203408d248b0b327bfa99a1bac16-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 27 COLUMNS
At line 164 RHS
At line 187 BOUNDS
At line 216 ENDATA
Problem MODEL has 22 rows, 24 columns and 76 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 12 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 12 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 12 strengthened rows, 0 substitutions
Cgl0004I processed model has 22 rows, 24 columns (20 intege